In [1]:
import pandas as pd


Data Path 

In [2]:
folder = f"Dataset\After-PreProcess\predict-by-rating"

In [3]:
defenders_df = pd.read_csv(folder+"/defenders_df.csv")
forward_df = pd.read_csv(folder+"/forward_df.csv")
gk_df = pd.read_csv(folder+"/gk_df.csv")
midfielders_df = pd.read_csv(folder+"/midfielders_df.csv")


Functions

In [4]:
def drop_not_needed_fields(df , position_group):
    if position_group == "defenders":
        df = df.drop(['Unnamed: 0', 'Next_Year_Club','GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position',  'Name' , 'Club', "Work Rate", "Positioning", "Position",
                     "Height", "Weight" , "Next_Year_Club"],axis = 1)
    elif position_group == "midfielders":
        df = df.drop(['Unnamed: 0', 'Next_Year_Club','GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position',  'Name' , 'Club', "Work Rate", "Positioning", "Position",
                     "Height", "Weight" , "Next_Year_Club"],axis = 1)
    elif position_group == "forwards":
        df = df.drop(['Unnamed: 0', 'Next_Year_Club','GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position',  'Name' , 'Club', "Work Rate", "Positioning", "Position",
                     "Height", "Weight" , "Next_Year_Club"],axis = 1)   
    elif  position_group == "goalkeepers":
        df = df.drop(['Unnamed: 0', 'Next_Year_Club', 'Best Position',  'Name' , 'Club', "Work Rate", "Positioning", "Position",
                     "Height", "Weight" , "Next_Year_Club",
                     'Crossing', 'Finishing','HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
                      'FKAccuracy', 'LongPassing', 'BallControl', 'Agility', 'Reactions', 'Balance', 'ShotPower',
                       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
                       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
                       'StandingTackle', 'DefensiveAwareness'],axis = 1)
    return df

In [5]:
def split_df(df, test_size=0.25, random_state=42):
    """Splits a Pandas dataframe into train and test sets.

    Args:
    df: The Pandas dataframe to split.
    test_size: The size of the test set, as a fraction of the total dataset.
    random_state: The random state to use for shuffling the data.

    Returns:
    A tuple of train and test dataframes.
    """

    # Shuffle the data.
    df = df.sample(frac=1, random_state=random_state)

    # Split the data into train and test sets.
    train_size = int(len(df) * (1 - test_size))
    train_df = df[:train_size]
    test_df = df[train_size:]

    return train_df, test_df


In [85]:
def get_features(position_group):
    if position_group == "defenders":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness']
    elif position_group == "midfielders":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness']
    elif position_group == "forwards":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness']
    elif  position_group == "goalkeepers":
        return ['GKDiving', 'GKHandling','GKKicking', 'GKPositioning', 'GKReflexes','Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Acceleration','SprintSpeed']
        

In [79]:
import xgboost as xgb
import numpy as np

def train_model(df, model_type, features):
    le = LabelEncoder()
    df['Next_Overall'] = le.fit_transform(df['Next_Overall'])
    df['Overall'] = le.fit_transform(df['Overall'])
    
    train_df, test_df = split_df(df)
    
    y_train = train_df['Next_Overall']
    y_test = test_df['Next_Overall']
    
#     train_df = train_df[features].copy()
#     test_df = test_df[features].copy()
    
    X_train = train_df[features].copy()
    X_test = test_df[features].copy()


    # Create a SimpleImputer to handle missing values
    imputer = SimpleImputer(strategy='mean')

    # Impute missing values in training data
    X_train_imputed = imputer.fit_transform(X_train)

    # Impute missing values in testing data
    X_test_imputed = imputer.transform(X_test)

    if model_type == "Random Forest":
        model = RandomForestClassifier()
    elif model_type == "xgboost":
        model = xgb.XGBClassifier()
    else:
        model = KNeighborsClassifier()
    dtrain = xgb.DMatrix(X_train_imputed, label=y_train)
    model.fit(X_train_imputed, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_imputed)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Model: {model_type}, Accuracy: {accuracy}')

    # Add the predictions to the dataframe in a column
    test_df['predictions'] = y_pred
    test_df['Next_Overall'] = le.inverse_transform(test_df['Next_Overall'])
    test_df['Overall'] = le.inverse_transform(test_df['Overall'])
    test_df['predictions'] = le.inverse_transform(test_df['predictions'])

    return model, test_df


Analyzing results

In [98]:
def analyze_movement_direction(df):
    same_count = 0
    higher_count = 0
    lower_count = 0

    for _, row in df.iterrows():
        next_overall = row['Next_Overall']
        prediction = row['predictions']

        if prediction > next_overall:
            higher_count += 1
        elif prediction < next_overall:
            lower_count += 1
        else:
            same_count += 1

    result = {
        'Predicted Right': same_count,
        'Predicted Higher': higher_count,
        'Predicted Lower': lower_count
    }

    return result


In [94]:
def analyze_movement(df):
    total_count = len(df)
    move_up_count = 0
    move_down_count = 0
    pred_move_up_count = 0
    pred_move_down_count = 0
    pred_same = 0
    same = 0
    for _, row in df.iterrows():
        overall = row['Overall']
        next_overall = row['Next_Overall']
        prediction = row['predictions']

        if next_overall > overall:
            move_up_count += 1
            if prediction > overall:
                pred_move_up_count += 1
        elif next_overall < overall:
            move_down_count += 1
            if prediction < overall:
                pred_move_down_count += 1
        else:
            same += 1
            if prediction == overall:
                pred_same += 1

    result = {
        'Total Rows': total_count,
        'Moved Up': move_up_count,
        'Predicted Move Up': pred_move_up_count,
        'Moved Down': move_down_count,
        'Predicted Move Down': pred_move_down_count,
        'Stayed the Same': same,
        'Predicted to Stay the Same': pred_same
    }

    return result


In [101]:
def generate_prediction_report(df):
    diff_counts = {'Exactly': 0,'Off by 1': 0, 'Off by 2-3': 0, 'Off by 4 or more': 0}

    for _, row in df.iterrows():
        next_overall = row['Next_Overall']
        prediction = row['predictions']
        diff = abs(next_overall - prediction)

        if diff == 1:
            diff_counts['Off by 1'] += 1
        elif diff >= 2 and diff <= 3:
            diff_counts['Off by 2-3'] += 1
        elif diff >= 4:
            diff_counts['Off by 4 or more'] += 1
        else:
            diff_counts['Exactly'] += 1

    return diff_counts


In [100]:
def create_pie_chart(data_dict):
    keys = list(data_dict.keys())
    values = list(data_dict.values())

    plt.figure(figsize=(6, 6))
    plt.pie(values, labels=keys, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    plt.show()


defenders predictions

In [80]:
xgmodel_defenders , defenderswithpredictions = train_model(defenders_df.copy(), "Random Forest" , get_features("defenders"))

Model: Random Forest, Accuracy: 0.24560143626570916


In [102]:
generate_prediction_report(defenderswithpredictions)

{'Exactly': 1368,
 'Off by 1': 1939,
 'Off by 2-3': 1599,
 'Off by 4 or more': 664}

In [96]:
analyze_movement(defenderswithpredictions)

{'Total Rows': 5570,
 'Moved Up': 1273,
 'Predicted Move Up': 200,
 'Moved Down': 3332,
 'Predicted Move Down': 2693,
 'Stayed the Same': 965,
 'Predicted to Stay the Same': 198}

In [99]:
analyze_movement_direction(defenderswithpredictions)

{'Predicted Right': 1368, 'Predicted Higher': 1909, 'Predicted Lower': 2293}

In [81]:
defenderswithpredictions

,Unnamed: 0,ID,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,...,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,version,Next_Overall,Next_Year_Club,DefensiveAwareness,predictions
17973,8178,213151,C. Norman,24,61,64,Walsall,275000,2000,1.0,...,11.0,14.0,11.0,9.0,CB,2021,62,Newport County,58,60
10400,8957,233800,F. Bjørkan,19,64,72,FK Bodø/Glimt,950000,6000,1.0,...,13.0,10.0,12.0,9.0,LB,2019,68,FK Bodø/Glimt,61,63
1567,6519,210787,M. Dos Santos,26,76,77,SD Eibar,4600000,42000,1.0,...,14.0,6.0,13.0,12.0,CB,2017,74,CD Leganés,74,75
10298,8529,235627,O. Fane,24,63,69,Oldham Athletic,2300000,10000,1.0,...,11.0,13.0,11.0,7.0,CB,2019,62,Shrewsbury,61,62
1706,7075,208493,T. Tawatha,24,69,73,Eintracht Frankfurt,210000,3000,1.0,...,14.0,9.0,9.0,6.0,LWB,2017,69,Eintracht Frankfurt,64,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,15213,228191,J. Aspropotamitis,22,64,72,Central Coast Mariners,230000,2000,1.0,...,15.0,14.0,10.0,10.0,CB,2019,62,Western United FC,62,63
21575,12486,228290,J. Laporte,27,72,74,FC Lorient,2500000,13000,1.0,...,6.0,15.0,15.0,14.0,CB,2022,72,FC Lorient,74,70
5390,4725,201136,M. Colin,25,71,74,Birmingham City,6000000,1000,1.0,...,10.0,9.0,11.0,11.0,RB,2018,70,Birmingham City,66,70
860,3594,232244,S. Ascacibar,19,72,85,Estudiantes de La Plata,2600000,4000,1.0,...,10.0,5.0,14.0,9.0,RB,2017,75,VfB Stuttgart,74,74


Midfielders

In [82]:
xgmodel_mid, midsswithpredictions = train_model(midfielders_df.copy(), "Random Forest" , get_features("midfielders"))

Model: Random Forest, Accuracy: 0.26042133572389065


In [81]:
midsswithpredictions

,ID,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Crossing,...,Interceptions,Vision,Penalties,Composure,StandingTackle,SlidingTackle,version,Next_Overall,DefensiveAwareness,predictions
386,184255,26,75,76,925000,3000,2.0,3.0,2.0,48,...,40,63,71,74,46,32,2017,70,49,74
1628,223933,19,64,73,475000,500,1.0,2.0,2.0,36,...,33,46,64,55,30,25,2017,63,37,62
10762,183712,35,66,66,300000,800,1.0,3.0,2.0,38,...,39,49,67,59,33,28,2021,67,28,64
11912,237712,22,70,78,3600000,5000,1.0,3.0,3.0,54,...,26,66,50,67,34,39,2022,76,30,68
7718,208159,24,72,75,1500000,7000,1.0,3.0,3.0,45,...,26,62,62,73,20,16,2020,69,35,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,213694,26,69,69,1500000,8000,1.0,2.0,3.0,64,...,36,68,58,54,36,33,2022,68,27,67
5191,200922,29,68,68,1000000,3000,2.0,3.0,3.0,62,...,42,58,59,62,30,27,2019,66,42,67
5390,178091,28,74,74,3400000,500,2.0,3.0,2.0,50,...,21,62,66,71,25,20,2019,75,35,72
860,197941,27,68,68,10000000,63000,1.0,4.0,3.0,40,...,24,48,69,56,23,26,2017,66,34,66


In [83]:
xgmodel_fw, fwwithpredictions = train_model(forward_df.copy(), "Random Forest" , get_features("forwards"))

Model: Random Forest, Accuracy: 0.24851330203442878


In [83]:
fwwithpredictions

,ID,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Crossing,...,Interceptions,Vision,Penalties,Composure,StandingTackle,SlidingTackle,version,Next_Overall,DefensiveAwareness,predictions
386,184255,26,75,76,925000,3000,2.0,3.0,2.0,48,...,40,63,71,74,46,32,2017,70,49,74
1628,223933,19,64,73,475000,500,1.0,2.0,2.0,36,...,33,46,64,55,30,25,2017,63,37,62
10762,183712,35,66,66,300000,800,1.0,3.0,2.0,38,...,39,49,67,59,33,28,2021,67,28,64
11912,237712,22,70,78,3600000,5000,1.0,3.0,3.0,54,...,26,66,50,67,34,39,2022,76,30,68
7718,208159,24,72,75,1500000,7000,1.0,3.0,3.0,45,...,26,62,62,73,20,16,2020,69,35,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,213694,26,69,69,1500000,8000,1.0,2.0,3.0,64,...,36,68,58,54,36,33,2022,68,27,67
5191,200922,29,68,68,1000000,3000,2.0,3.0,3.0,62,...,42,58,59,62,30,27,2019,66,42,67
5390,178091,28,74,74,3400000,500,2.0,3.0,2.0,50,...,21,62,66,71,25,20,2019,75,35,72
860,197941,27,68,68,10000000,63000,1.0,4.0,3.0,40,...,24,48,69,56,23,26,2017,66,34,66


Goalkeepers

In [31]:
type(gk_df.columns)

pandas.core.indexes.base.Index

In [86]:
xgmodel_gk, gkwithpredictions = train_model(gk_df.copy(), "Random Forest" , get_features("goalkeepers"))

Model: Random Forest, Accuracy: 0.3046875


## Model With Goals

In [32]:
g_folder = f"Dataset\After-PreProcess\predict-with-real-life-stats"

In [52]:
g_fielders = pd.read_csv(g_folder+"/stats_Full_df.csv")
g_keepers = pd.read_csv(g_folder+"/stats_gk_df.csv")

In [54]:
g_defenders = pd.read_csv(g_folder+"/stats_defenders_df.csv")

In [73]:
stats_forward_df = pd.read_csv(g_folder+"/stats_forward_df.csv")

In [34]:
g_fielders.columns

Index(['ID', 'Name', 'Age', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate',
       'Position', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'version',
       'Next_Overall', 'Next_Year_Club', 'DefensiveAwareness',
       'Name_Adjusted_x', 'First Initial_x', 'Last Name', 'Rk', 'Player',
       'Nation', 'Pos', 'Squad', 'Comp', 'Born', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR'

In [66]:
def get_features_with_goals(position_group):
    if position_group == "defenders":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG',
       'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1',
       'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1',
       'npxG+xAG.1']
    elif position_group == "midfielders":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG',
       'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1',
       'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1',
       'npxG+xAG.1']
    elif position_group == "forwards":
        return ['Age', 'Overall', 'Potential' , 'Value', 'Wage',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'StandingTackle', 'SlidingTackle','DefensiveAwareness', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG',
       'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1',
       'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1',
       'npxG+xAG.1']
    elif  position_group == "goalkeepers":
        return


In [72]:
xgmodel_defenders_g , defenderswithpredictions_g = train_model(g_defenders.copy(), "Random Forest", get_features_with_goals("defenders"))

Model: Random Forest, Accuracy: 0.2088607594936709


In [74]:
xgmodel_forwards_g , forwardswithpredictions_g = train_model(stats_forward_df.copy(), "Random Forest" , get_features_with_goals("defenders"))

Model: Random Forest, Accuracy: 0.2670807453416149
